# 서울 구별 주택공급 현황 상세 분석

이 노트북은 실제 주택공급 데이터에서 연도별, 월별 상세 정보를 분석합니다.

## 분석 내용
1. 데이터 로드 및 구조 파악
2. 서울 25개 구 필터링
3. 연도별 공급 추이
4. 월별 공급 패턴
5. 구별 상세 분석
6. 시계열 그래프

## 1. 환경 설정

In [ ]:
import sys
sys.path.append("..")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from pathlib import Path
from datetime import datetime

# 한글 폰트 설정 (Windows)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 그래프 스타일 설정
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✅ 환경 설정 완료")

## 2. 데이터 로드

In [ ]:
# 실제 데이터 파일 경로
REAL_DATA_FILE = "주택종류별+주택-+읍면동(연도+끝자리+0,5),+시군구(그+외+연도)_20251028224837.xlsx"

print(f"📂 데이터 파일 로드 중: {REAL_DATA_FILE}")
df = pd.read_excel(REAL_DATA_FILE)

print(f"\n✅ 데이터 로드 완료!")
print(f"   - 총 행 수: {len(df):,}건")
print(f"   - 총 열 수: {len(df.columns)}개")
print(f"\n📋 컬럼 목록:")
for i, col in enumerate(df.columns, 1):
    print(f"   {i}. {col}")
    
print(f"\n📊 처음 5행 미리보기:")
df.head()

## 3. 컬럼 매핑 및 전처리

⚠️ **중요:** 실제 데이터의 컬럼 이름에 맞게 아래 변수를 수정하세요!

In [ ]:
# TODO: 실제 컬럼 이름에 맞게 수정하세요
# 위의 "컬럼 목록"을 보고 실제 컬럼 이름을 입력하세요

# 예시:
DATE_COL = "연월"  # 또는 "날짜", "기준월" 등
DISTRICT_COL = "시군구"  # 또는 "지역", "행정구역" 등
VALUE_COL = "주택"  # 또는 "공급량", "세대수" 등

print("📌 선택한 컬럼:")
print(f"   - 날짜: {DATE_COL}")
print(f"   - 구: {DISTRICT_COL}")
print(f"   - 값: {VALUE_COL}")

# 컬럼 존재 여부 확인
for col_name in [DATE_COL, DISTRICT_COL, VALUE_COL]:
    if col_name not in df.columns:
        print(f"❌ 경고: '{col_name}' 컬럼이 없습니다. 위의 컬럼 목록을 확인하고 수정하세요!")
    else:
        print(f"✅ '{col_name}' 컬럼 확인")

### 3.1 날짜 데이터 변환

In [ ]:
# 날짜 컬럼 변환
if DATE_COL in df.columns:
    # 날짜 형식 자동 인식 시도
    try:
        df[DATE_COL] = pd.to_datetime(df[DATE_COL])
        print(f"✅ 날짜 변환 성공!")
    except:
        # 형식 지정 시도 (예: '202401' 형식)
        try:
            df[DATE_COL] = pd.to_datetime(df[DATE_COL], format='%Y%m')
            print(f"✅ 날짜 변환 성공 (YYYYMM 형식)!")
        except:
            print(f"❌ 날짜 변환 실패. 수동으로 format 지정이 필요합니다.")
            print(f"샘플 값: {df[DATE_COL].iloc[0]}")

    # 연도, 월 컬럼 추가
    df['연도'] = df[DATE_COL].dt.year
    df['월'] = df[DATE_COL].dt.month
    
    print(f"\n📅 날짜 범위:")
    print(f"   시작: {df[DATE_COL].min()}")
    print(f"   종료: {df[DATE_COL].max()}")
    print(f"   총 기간: {(df[DATE_COL].max() - df[DATE_COL].min()).days // 365}년")

## 4. 서울 25개 구 필터링

In [ ]:
from tools.housing_supply_tool import SEOUL_DISTRICTS

print("📍 서울 25개 구 목록:")
for i, district in enumerate(SEOUL_DISTRICTS, 1):
    print(f"   {i:2d}. {district}", end="  ")
    if i % 5 == 0:
        print()

# 서울 구만 필터링
if DISTRICT_COL in df.columns:
    seoul_df = df[
        df[DISTRICT_COL].apply(
            lambda x: any(district in str(x) for district in SEOUL_DISTRICTS)
        )
    ].copy()
    
    print(f"\n\n✅ 필터링 완료!")
    print(f"   - 필터링 전: {len(df):,}건")
    print(f"   - 필터링 후: {len(seoul_df):,}건")
    print(f"   - 포함된 구: {seoul_df[DISTRICT_COL].nunique()}개")
    
    # 포함된 구 목록
    found_districts = seoul_df[DISTRICT_COL].unique()
    print(f"\n📌 데이터에 포함된 구:")
    for district in sorted(found_districts):
        count = len(seoul_df[seoul_df[DISTRICT_COL] == district])
        print(f"   - {district}: {count:,}건")

## 5. 연도별 분석

In [ ]:
# 연도별 공급량 집계
if '연도' in seoul_df.columns and VALUE_COL in seoul_df.columns:
    yearly_supply = seoul_df.groupby('연도')[VALUE_COL].sum().reset_index()
    yearly_supply = yearly_supply.sort_values('연도')
    
    print("📊 연도별 공급량 (서울 전체):")
    print(yearly_supply.to_string(index=False))
    
    # 그래프
    plt.figure(figsize=(14, 6))
    plt.bar(yearly_supply['연도'], yearly_supply[VALUE_COL], color='skyblue', edgecolor='navy')
    plt.title('서울시 연도별 주택공급량 추이', fontsize=16, fontweight='bold')
    plt.xlabel('연도', fontsize=12)
    plt.ylabel('공급량', fontsize=12)
    plt.grid(axis='y', alpha=0.3)
    
    # 값 표시
    for idx, row in yearly_supply.iterrows():
        plt.text(row['연도'], row[VALUE_COL], f"{row[VALUE_COL]:,.0f}", 
                ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.show()

## 6. 월별 분석

In [ ]:
# 월별 공급량 패턴 (전체 기간 평균)
if '월' in seoul_df.columns and VALUE_COL in seoul_df.columns:
    monthly_pattern = seoul_df.groupby('월')[VALUE_COL].mean().reset_index()
    monthly_pattern = monthly_pattern.sort_values('월')
    
    print("📊 월별 평균 공급량 (전체 기간):")
    print(monthly_pattern.to_string(index=False))
    
    # 그래프
    plt.figure(figsize=(14, 6))
    plt.plot(monthly_pattern['월'], monthly_pattern[VALUE_COL], marker='o', 
            linewidth=2, markersize=8, color='coral')
    plt.title('월별 평균 주택공급량 패턴', fontsize=16, fontweight='bold')
    plt.xlabel('월', fontsize=12)
    plt.ylabel('평균 공급량', fontsize=12)
    plt.xticks(range(1, 13))
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 7. 연도별 × 월별 교차 분석

In [ ]:
# 연도별, 월별 집계
if '연도' in seoul_df.columns and '월' in seoul_df.columns and VALUE_COL in seoul_df.columns:
    year_month_supply = seoul_df.groupby(['연도', '월'])[VALUE_COL].sum().reset_index()
    
    # 피벗 테이블 생성
    pivot_table = year_month_supply.pivot(index='연도', columns='월', values=VALUE_COL)
    
    print("📊 연도별 × 월별 공급량 교차표:")
    print(pivot_table.to_string())
    
    # 히트맵
    plt.figure(figsize=(16, 8))
    plt.imshow(pivot_table.values, aspect='auto', cmap='YlOrRd', interpolation='nearest')
    plt.colorbar(label='공급량')
    plt.title('연도별 × 월별 주택공급량 히트맵', fontsize=16, fontweight='bold')
    plt.xlabel('월', fontsize=12)
    plt.ylabel('연도', fontsize=12)
    plt.xticks(range(len(pivot_table.columns)), pivot_table.columns)
    plt.yticks(range(len(pivot_table.index)), pivot_table.index)
    plt.tight_layout()
    plt.show()

## 8. 구별 상세 분석

### 8.1 구별 총 공급량 순위

In [ ]:
# 구별 총 공급량
if DISTRICT_COL in seoul_df.columns and VALUE_COL in seoul_df.columns:
    district_total = seoul_df.groupby(DISTRICT_COL)[VALUE_COL].sum().reset_index()
    district_total = district_total.sort_values(VALUE_COL, ascending=False)
    district_total['순위'] = range(1, len(district_total) + 1)
    
    print("🏆 구별 총 공급량 순위:")
    print(district_total[[' 순위', DISTRICT_COL, VALUE_COL]].to_string(index=False))
    
    # 상위 10개 구 그래프
    top10 = district_total.head(10)
    
    plt.figure(figsize=(14, 8))
    bars = plt.barh(top10[DISTRICT_COL], top10[VALUE_COL], color='teal', edgecolor='darkblue')
    plt.title('구별 총 공급량 TOP 10', fontsize=16, fontweight='bold')
    plt.xlabel('총 공급량', fontsize=12)
    plt.ylabel('구', fontsize=12)
    plt.gca().invert_yaxis()
    
    # 값 표시
    for idx, (bar, row) in enumerate(zip(bars, top10.iterrows())):
        plt.text(row[1][VALUE_COL], idx, f" {row[1][VALUE_COL]:,.0f}", 
                va='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()

### 8.2 특정 구의 연도별 추이

In [ ]:
# 원하는 구를 선택하세요
TARGET_DISTRICT = "강남구"  # 🔧 여기를 수정하여 다른 구를 확인하세요

# 해당 구의 데이터 필터링
district_data = seoul_df[seoul_df[DISTRICT_COL].str.contains(TARGET_DISTRICT, na=False)]

if len(district_data) > 0 and '연도' in district_data.columns:
    district_yearly = district_data.groupby('연도')[VALUE_COL].sum().reset_index()
    
    print(f"📊 {TARGET_DISTRICT} 연도별 공급량:")
    print(district_yearly.to_string(index=False))
    
    # 그래프
    plt.figure(figsize=(14, 6))
    plt.plot(district_yearly['연도'], district_yearly[VALUE_COL], 
            marker='o', linewidth=3, markersize=10, color='darkgreen')
    plt.title(f'{TARGET_DISTRICT} 연도별 주택공급량 추이', fontsize=16, fontweight='bold')
    plt.xlabel('연도', fontsize=12)
    plt.ylabel('공급량', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print(f"❌ '{TARGET_DISTRICT}' 데이터를 찾을 수 없습니다.")

### 8.3 특정 구의 월별 패턴

In [ ]:
# 위에서 선택한 구의 월별 패턴
if len(district_data) > 0 and '월' in district_data.columns:
    district_monthly = district_data.groupby('월')[VALUE_COL].mean().reset_index()
    
    print(f"📊 {TARGET_DISTRICT} 월별 평균 공급량:")
    print(district_monthly.to_string(index=False))
    
    # 그래프
    plt.figure(figsize=(14, 6))
    plt.bar(district_monthly['월'], district_monthly[VALUE_COL], 
           color='orange', edgecolor='darkred')
    plt.title(f'{TARGET_DISTRICT} 월별 평균 주택공급량', fontsize=16, fontweight='bold')
    plt.xlabel('월', fontsize=12)
    plt.ylabel('평균 공급량', fontsize=12)
    plt.xticks(range(1, 13))
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()

## 9. 여러 구 비교 분석

In [ ]:
# 비교할 구 목록 (🔧 원하는 구로 수정하세요)
COMPARE_DISTRICTS = ["강남구", "서초구", "송파구", "강동구"]

# 각 구의 연도별 데이터
plt.figure(figsize=(16, 8))

for district in COMPARE_DISTRICTS:
    district_data = seoul_df[seoul_df[DISTRICT_COL].str.contains(district, na=False)]
    if len(district_data) > 0 and '연도' in district_data.columns:
        yearly_data = district_data.groupby('연도')[VALUE_COL].sum().reset_index()
        plt.plot(yearly_data['연도'], yearly_data[VALUE_COL], 
                marker='o', linewidth=2, markersize=6, label=district)

plt.title('구별 연도별 주택공급량 비교', fontsize=16, fontweight='bold')
plt.xlabel('연도', fontsize=12)
plt.ylabel('공급량', fontsize=12)
plt.legend(fontsize=11, loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 10. 전체 시계열 추이

In [ ]:
# 날짜별 전체 공급량
if DATE_COL in seoul_df.columns and VALUE_COL in seoul_df.columns:
    timeseries = seoul_df.groupby(DATE_COL)[VALUE_COL].sum().reset_index()
    timeseries = timeseries.sort_values(DATE_COL)
    
    # 이동평균 추가 (6개월)
    timeseries['이동평균_6개월'] = timeseries[VALUE_COL].rolling(window=6).mean()
    
    plt.figure(figsize=(16, 8))
    plt.plot(timeseries[DATE_COL], timeseries[VALUE_COL], 
            alpha=0.5, linewidth=1, label='실제 공급량', color='lightblue')
    plt.plot(timeseries[DATE_COL], timeseries['이동평균_6개월'], 
            linewidth=3, label='6개월 이동평균', color='darkblue')
    
    plt.title('서울시 전체 주택공급량 시계열 추이', fontsize=16, fontweight='bold')
    plt.xlabel('날짜', fontsize=12)
    plt.ylabel('공급량', fontsize=12)
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 11. 결과 저장

In [ ]:
# 분석 결과를 Excel로 저장
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

with pd.ExcelWriter(f'서울_공급현황_상세분석_{timestamp}.xlsx') as writer:
    # 시트 1: 원본 데이터 (서울만)
    seoul_df.to_excel(writer, sheet_name='서울_원본데이터', index=False)
    
    # 시트 2: 연도별 집계
    if 'yearly_supply' in locals():
        yearly_supply.to_excel(writer, sheet_name='연도별_공급량', index=False)
    
    # 시트 3: 구별 총계
    if 'district_total' in locals():
        district_total.to_excel(writer, sheet_name='구별_총공급량', index=False)
    
    # 시트 4: 연도×월 교차표
    if 'pivot_table' in locals():
        pivot_table.to_excel(writer, sheet_name='연도월별_교차표')

print(f"✅ 결과 저장 완료: 서울_공급현황_상세분석_{timestamp}.xlsx")

## 12. 분석 요약

In [ ]:
print("="*80)
print("📊 서울시 주택공급 현황 분석 요약")
print("="*80)

if VALUE_COL in seoul_df.columns:
    total_supply = seoul_df[VALUE_COL].sum()
    avg_supply = seoul_df[VALUE_COL].mean()
    max_supply = seoul_df[VALUE_COL].max()
    min_supply = seoul_df[VALUE_COL].min()
    
    print(f"\n📈 전체 통계:")
    print(f"   - 총 공급량: {total_supply:,.0f}")
    print(f"   - 평균 공급량: {avg_supply:,.2f}")
    print(f"   - 최대 공급량: {max_supply:,.0f}")
    print(f"   - 최소 공급량: {min_supply:,.0f}")

if DATE_COL in seoul_df.columns:
    print(f"\n📅 기간:")
    print(f"   - 시작: {seoul_df[DATE_COL].min().strftime('%Y-%m')}")
    print(f"   - 종료: {seoul_df[DATE_COL].max().strftime('%Y-%m')}")
    print(f"   - 총 기간: {(seoul_df[DATE_COL].max() - seoul_df[DATE_COL].min()).days // 30}개월")

if DISTRICT_COL in seoul_df.columns:
    print(f"\n🏘️ 지역:")
    print(f"   - 분석 구역: 서울시 {seoul_df[DISTRICT_COL].nunique()}개 구")
    
if 'district_total' in locals():
    top_3 = district_total.head(3)
    print(f"\n🏆 TOP 3 공급량 많은 구:")
    for idx, row in top_3.iterrows():
        print(f"   {row['순위']}. {row[DISTRICT_COL]}: {row[VALUE_COL]:,.0f}")

print("\n" + "="*80)
print("✅ 분석 완료!")
print("="*80)